# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil editar e modificar um pdf
- Mesmo assim o Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [1]:
import PyPDF2 as pyf
from pathlib import Path

arquivo_pdf = pyf.PdfReader(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')

i = 1
for pagina in arquivo_pdf.pages:
    arquivo_novo = pyf.PdfWriter()
    arquivo_novo.add_page(pagina)
    with Path(fr'Documentos PDF - Mentoria\Páginas Separadas\Arquivo{i}.pdf').open(mode='wb') as arquivo_final:
        arquivo_novo.write(arquivo_final)
        i += 1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [2]:
paginas = [1, 14, 16]

novo_pdf = pyf.PdfWriter()
for num_pagina in paginas:
    arquivo_pdf = pyf.PdfReader(fr'Documentos PDF - Mentoria\Páginas Separadas\Arquivo{num_pagina}.pdf')
    pagina = arquivo_pdf.pages[0]
    novo_pdf.add_page(pagina)

with Path(r'Documentos PDF - Mentoria\Consolidado.pdf').open(mode='wb') as arquivo_final:
        novo_pdf.write(arquivo_final)

### Extra: Para adicionar todas as páginas de 2 pdfs

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

- Rodar Página

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [ ]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url